
# Getting the model


In [0]:
import torch
model = torch.hub.load('pytorch/vision:v0.5.0', 'fcn_resnet101', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(N, 3, H, W)`, where `N` is the number of images, `H` and `W` are expected to be at least `224` pixels.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

The model returns an `OrderedDict` with two Tensors that are of the same height and width as the input Tensor, but with 21 classes.
`output['out']` contains the semantic masks, and `output['aux']` contains the auxillary loss values per-pixel. In inference mode, `output['aux']` is not useful.
So, `output['out']` is of shape `(N, 21, H, W)`. More documentation can be found [here](https://pytorch.org/docs/stable/torchvision/models.html#object-detection-instance-segmentation-and-person-keypoint-detection).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### To load the data


In [0]:
import numpy as np
import pandas as pd

In [0]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils

In [0]:
##here put your custom destination folder
#arr_train_labels = np.load('drive/My Drive/Dataset_test_fcn')
#arr_val_labels = np.load('drive/My Drive/Lab2_DL/val_labels.npy')

### Input


In [0]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Third Branch/Magrabia/Magrabia/MagrabiaMale"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "fcn_resnet101"

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

### Model Training and Validation Code

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

### Set Model Parameters, unfreezing layers

In [0]:
def initialize_model(model_name, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_fcn = torch.hub.load('pytorch/vision:v0.5.0', 'fcn_resnet101', pretrained=True)
    set_parameter_requires_grad(model_fcn, feature_extract)
    num_ftrs = model_fcn.aux_classifier[4]
    #Here i put an output of the last layer of 2 because we want to segment into two parts, it was 21 before.
    model_fcn.backbone.layer4[2].conv3 = nn.Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    model_fcn.classifier[0] =  nn.Conv2d(2048, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model_fcn.classifier[4]= nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
    #model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

    return model_fcn, input_size

# Initialize the model for this run
model_fcn, input_size = initialize_model(model_name, feature_extract, use_pretrained=True)
model_fcn2, input_size = initialize_model(model_name, feature_extract, use_pretrained=True)

# Print the model we just instantiated
#print(model_fcn)

# Loading the data




### Preprocess images



In [0]:
import glob
import re

from PIL import Image
import cv2


In [0]:
from collections import OrderedDict
camvid_colors = OrderedDict([
    ("main", np.array([13, 240,231], dtype=np.uint8)),
    ("in", np.array([1,62,231], dtype=np.uint8)),
    ("out", np.array([0, 0, 0], dtype=np.uint8))])

In [0]:
from scipy import stats

def convert_label_to_grayscale(img):
    im = np.float32(img)
    """ 
    for i in range(im.shape[0]) :
      for j in range(im.shape[1]) :
        rgb = im[i][j]
        if (not (rgb == [0,0,0]).all() ) and (not (rgb == [1,62,231]).all() ):
          print(rgb)
    """
    #print("im",np.unique(im))
    #print(im.shape)
    out = (np.ones(im.shape[:2]) * 2).astype(np.uint8)
    for gray_val, (label, rgb) in enumerate(camvid_colors.items()):
        #print(im == np.asarray(rgb) )
        match_pxls = np.where((im == np.asarray(rgb)).sum(-1) == 3)
        #print(match_pxls)
        if (gray_val == 0) :
          out[match_pxls] = 1
        else : 
          out[match_pxls] = 0
    assert (out != 255).all(), "rounding errors or missing classes in camvid_colors"
    return Image.fromarray(out.astype(np.uint8))


### Creating the dataset

In [0]:
class RIGA(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, input_dir, disc_dir, cup_dir, transform=None):

        discard_disc = [25,15,16,23,27,28,29,39,46]
        omit_disc = []
        omit_disc.append([22,47]) #1
        omit_disc.append([22,47]) #2
        omit_disc.append([22]) #3
        omit_disc.append([8,11,22]) #4
        omit_disc.append([8,11,22]) #5
        omit_disc.append([8,11]) #6
        self.input_dir = input_dir
        filenames = glob.glob(input_dir+'/*prime.tif')
        self.images_paths = []
        self.disc_paths = []
        self.cup_paths = []
        for image in glob.glob(input_dir+'/*prime.tif') : 
          img_id = [int(n) for n in re.findall(r'\d+', image)][0]
          if img_id not in discard_disc : 
            for disc in glob.glob(disc_dir+'/*.png') :
              #print(disc) 
              disc_idx = [int(n) for n in re.findall(r'\d+', disc)]
              if (disc_idx[0] not in omit_disc[disc_idx[1]-1]) : 
                for cup in  glob.glob(cup_dir+'/*.png') :
                  cup_idx = [int(n) for n in re.findall(r'\d+', cup)]
                  if(cup_idx[0]==disc_idx[0]) and (cup_idx[1]==disc_idx[1]) and (cup_idx[0]==img_id) : 
                    self.images_paths.append(image)
                    self.disc_paths.append(disc)
                    self.cup_paths.append(cup)


        i=0
        print(len(self.images_paths))
        print(len(self.disc_paths))
        print(len(self.cup_paths))
        self.transform = transform

    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #print(self.images_paths[idx])
        image = Image.open(self.images_paths[idx])
        disc = Image.open(self.disc_paths[idx])
        cup = Image.open(self.cup_paths[idx])
        sample = {'image': image, 'disc' : disc , 'cup' : cup}
        disc = convert_label_to_grayscale(disc)
        cup = convert_label_to_grayscale(cup)
        #print(np.float32(disc))
        #print(np.float32(disc).max(),np.float32(cup).max())
        if self.transform:
            image = self.transform(image)
        data_transforms =transforms.Compose([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

        disc = torch.from_numpy(cv2.resize(np.float32(disc), 
                                           dsize=(image.size()[2],image.size()[1]), 
                                           interpolation=cv2.INTER_NEAREST)).long() #image.size()[1],image.size()[2]
        cup =torch.from_numpy(cv2.resize(np.float32(cup) ,
                                         dsize=(image.size()[2],image.size()[1]), 
                                         interpolation=cv2.INTER_NEAREST)).long()
        #print(np.float32(disc).max(),np.float32(cup).max())
        image = data_transforms(image)
        #print(np.float32(disc))
        return image, disc,cup

### Create dataset, train and validation set, and preprocess the data

In [0]:

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms =transforms.Compose([
        #transforms.ToTensor(),
        torchvision.transforms.Resize((input_size)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



print("Initializing Datasets and Dataloaders...")

disc_dir = "/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Third Branch/Magrabia/outputs/disc_segmented"
cup_dir = "/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Third Branch/Magrabia/outputs/cup_segmented"
# Create training and validation datasets
image_dataset = RIGA(data_dir,disc_dir,cup_dir, data_transforms)
print(len(image_dataset))

train_set, val_set = random_split(image_dataset, [200,15])
# Create training and validation dataloaders
dataloaders_dict = DataLoader(train_set, batch_size=batch_size, shuffle=True)

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...
215
215
215
215


### Check unfreezed layers and intialize optimizer

In [0]:
model_fcn = model_fcn.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_fcn.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_fcn.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_fcn.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=0.001)

Params to learn:
	 backbone.layer4.2.conv3.weight
	 classifier.0.weight
	 classifier.4.weight


In [0]:
model_fcn2 = model_fcn2.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update2 = model_fcn2.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update2 = []
    for name,param in model_fcn2.named_parameters():
        if param.requires_grad == True:
            params_to_update2.append(param)
            print("\t",name)
else:
    for name,param in model_fcn2.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer2 = optim.Adam(params_to_update2, lr=0.001)

Params to learn:
	 backbone.layer4.2.conv3.weight
	 classifier.0.weight
	 classifier.4.weight


### Define the 2-D Cross Entropy Loss Function 

In [0]:
import torch.nn.functional as F
from distutils.version import LooseVersion
def cross_entropy2d(input, target,device, weight=None, size_average=True):
    # input: (n, c, h, w), target: (n, h, w)
    n, c, h, w = input.size()
    # log_p: (n, c, h, w)
    if LooseVersion(torch.__version__) < LooseVersion('0.3'):
        # ==0.2.X
        log_p = F.log_softmax(input)
    else:
        # >=0.3
        log_p = F.log_softmax(input, dim=1)
    # log_p: (n*h*w, c)
    log_p = log_p.transpose(1, 2).transpose(2, 3).contiguous()
    log_p = log_p[target.view(n, h, w, 1).repeat(1, 1, 1, c) >= 0]
    log_p = log_p.view(-1, c)
    # target: (n*h*w,)
    mask = target >= 0
    target = target[mask]
    loss = F.nll_loss(log_p.to(device), target.to(device), weight=weight, reduction='sum')
    if size_average:
        loss /= mask.data.sum()
    return loss

### Train the disc FCN

In [0]:
def train_model_disc(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for (inputs, discs, cups) in dataloaders:
            inputs = inputs.to(device)
            dics = discs.to(device)
            cups = discs.to(device)
#.permute(0,3,1,2)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                # Get model outputs and calculate loss
                # Special case for inception because in training it has an auxiliary output. In train
                #   mode we calculate the loss by summing the final output and the auxiliary output
                #   but in testing we only consider the final output
                outputs = model(inputs.float())
                #print(type(outputs))
                outputs = outputs["out"]
                outputs =  outputs.to(device)
                """for k,v in outputs.items() :
                    print
                    output_disc[k] = v[:3, :, :] 
                    output_cups[k] = v[3:, :, :]""" 
                #print(output_disc.size(),output_cups.size())
                loss = cross_entropy2d(outputs, discs,device)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloaders.dataset)

        print('Loss: {:.4f}'.format(epoch_loss))

        if epoch_loss < 0.001 :
          break 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, running_loss


### Train the Cup FCN

In [0]:
def train_model_cup(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for (inputs, discs, cups) in dataloaders:
            inputs = inputs.to(device)
            dics = discs.to(device)
            cups = discs.to(device)
#.permute(0,3,1,2)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                # Get model outputs and calculate loss
                # Special case for inception because in training it has an auxiliary output. In train
                #   mode we calculate the loss by summing the final output and the auxiliary output
                #   but in testing we only consider the final output
                outputs = model(inputs.float())
                #print(type(outputs))
                outputs = outputs["out"]
                outputs =  outputs.to(device)
                """for k,v in outputs.items() :
                    print
                    output_disc[k] = v[:3, :, :] 
                    output_cups[k] = v[3:, :, :]""" 
                #print(output_disc.size(),output_cups.size())
                loss = cross_entropy2d(outputs, cups,device)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloaders.dataset)

        print('Loss: {:.4f}'.format(epoch_loss))

        if epoch_loss < 0.001 :
          break 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, running_loss

In [0]:
criterion = None #nn.NLLLoss2d() #nn.CrossEntropyLoss()
# Train and evaluate
model_fcn = model_fcn.float()
model_fcn, hist = train_model_disc(model_fcn, dataloaders_dict, criterion, optimizer, num_epochs=20)


In [0]:
model_fcn2 = model_fcn2.float()
model_fcn2, hist = train_model_cup(model_fcn2, dataloaders_dict, criterion, optimizer2, num_epochs=10)

### Save the model 

In [0]:
PATH = "FCN.model"
PATH2 = "FCN2.model"

In [0]:
torch.save(model_fcn.state_dict(), PATH)
torch.save(model_fcn2.state_dict(), PATH2)

In [0]:
model_fcn.load_state_dict(torch.load(PATH))
model_fcn2.load_state_dict(torch.load(PATH2))
model_fcn.eval()
model_fcn2.eval()

### Evaluate the MSE for the ratio on the model 

In [0]:
err = []
mse = 0
for (im,d,c) in val_set:
  im = im.to(device)
  d = d.to(device)
  c = c.to(device)
  with torch.no_grad():
    boutput = model_fcn(im.unsqueeze(0))
    output = boutput["out"][0].argmax(0).float()
  with torch.no_grad():
    boutput2 = model_fcn2(im.unsqueeze(0))
    output2 = boutput2["out"][0].argmax(0).float()
  r_pred = output2.sum()/output.sum()
  r = c.sum()/d.sum()
  print(r,r_pred,output2.sum(),output.sum(),c.sum(),d.sum())
  err += [(r_pred-r)**2]
for (im,d,c) in train_set :
  im = im.to(device)
  d = d.to(device)
  c = c.to(device)
  with torch.no_grad():
    boutput = model_fcn(im.unsqueeze(0))
    output = boutput["out"][0].argmax(0).float()
  with torch.no_grad():
    boutput2 = model_fcn2(im.unsqueeze(0))
    output2 = boutput2["out"][0].argmax(0).float()
  r_pred = output2.sum()/output.sum()
  r = c.sum()/d.sum()
  print(r,r_pred,output2.sum(),output.sum(),c.sum(),d.sum())
  err += [(r_pred-r)**2]

In [0]:
print(sum(err)/len(err))

tensor(0.8898, device='cuda:0')
